# Agrupamiento de Niveles
En el siguiente notebook se va a agrupar ciertos niveles de las variables categóricas con una frecuencia baja, el tratamiento será en las siguientes variables:
- Parentesco con el jefe del hogar
- ETNIA
- DISCAPACIDADES
- FINANCIEROS

In [2]:
# Carga de librerias
import pandas as pd
import os

In [3]:
# Carga del conjunto de datos consolidado
path= r"C:\Users\andre\OneDrive\Escritorio\Proyecto de Grado\result\db_consolidado_procesado"
month= 'Enero'
ruta_carga= os.path.join(path, month + '_Procesado.csv')
data= pd.read_csv(ruta_carga, sep= ';', encoding= 'latin-1')

In [4]:
data.head()

,DIRECTORIO,ZONA_TERRITORIAL,FACTOR_EXPANSION,DPTO,SEXO,EDAD,PARENTESCO_JEFE_DE_HOGAR,MADRE_RESIDE_HOGAR,PADRE_RESIDE_HOGAR,SE_CONSIDERA_CAMPESINO,...,HOGAR_TIENE_CUENTA_AHORROS,HOGAR_TIENE_CDT,HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA,HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO,HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION,HOGAR_TIENE_TARJETA_CREDITO,HOGAR_TIENE_OTRO_FINANCIERO,HOGAR_TIENE_NINGUNO_DE_ESTOS,HOGAR_NO_SABE_FINANCIERO,NUMERO_PERSONAS_HOGAR
0,7309060,Cabecera,1580.677273,CUNDINAMARCA,Masculino,27,"Pareja, esposo(a), conyuge, companero(a)",No,No,Si,...,Si,No,No,No,No,No,No,No,No,3
1,7309064,Resto,324.598262,RISALRALDA,Masculino,18,"Hijo(a), hijastro(a)",Si,No,Si,...,Si,No,No,No,No,No,No,No,No,5
2,7309067,Resto,167.457366,RISALRALDA,Femenino,25,"Hijo(a), hijastro(a)",No,Si,Si,...,Si,No,No,No,No,No,No,No,No,2
3,7309073,Resto,202.189536,RISALRALDA,Femenino,27,"Hijo(a), hijastro(a)",Si,No,Si,...,Si,No,No,No,No,No,No,No,No,6
4,7309073,Resto,202.189536,RISALRALDA,Femenino,23,"Hijo(a), hijastro(a)",Si,No,Si,...,Si,No,No,No,No,No,No,No,No,6


In [5]:
# Función que se realiza el cambio necesario
def replace_values(df, column, mapping):
    df[column] = df[column].replace(mapping)
    return df

## Parentesco Jefe de Hogar

In [6]:
data['PARENTESCO_JEFE_DE_HOGAR'].value_counts()

PARENTESCO_JEFE_DE_HOGAR
Hijo(a), hijastro(a)                                  3212
Jefe (a) del hogar                                    1804
Pareja, esposo(a), conyuge, companero(a)               749
Nieto(a)                                               339
Otro  pariente                                         244
Hermano(a) o hermanastro(a)                            186
Yerno o nuera                                          186
Otro no pariente                                       154
Empleado(a) del servicio domestico y sus parientes      17
Trabajador                                               5
Suegro(a)                                                3
Pensionista                                              2
Padre o madre                                            1
Name: count, dtype: int64

In [7]:
jefe = {
    'Padre o madre':'Otro pariente',
    'Suegro(a)':'Otro pariente',
    'Yerno o nuera':'Otro pariente',
    'Nieto(a)':'Otro pariente',
    'Otro  pariente':'Otro pariente',
    'Empleado(a) del servicio domestico y sus parientes':'No pariente',
    'Pensionista':'No pariente',
    'Trabajador':'No pariente',
    'Otro no pariente':'No pariente'
}

In [8]:
data['PARENTESCO_JEFE_DE_HOGAR'] = data['PARENTESCO_JEFE_DE_HOGAR'].replace(jefe)

In [9]:
data['PARENTESCO_JEFE_DE_HOGAR'].value_counts()

PARENTESCO_JEFE_DE_HOGAR
Hijo(a), hijastro(a)                        3212
Jefe (a) del hogar                          1804
Otro pariente                                773
Pareja, esposo(a), conyuge, companero(a)     749
Hermano(a) o hermanastro(a)                  186
No pariente                                  178
Name: count, dtype: int64

## ETNIA

In [10]:
data['ETNIA'].value_counts()

ETNIA
Ninguno de los anteriores           6161
Negro, mulato (afrodescendiente)     509
Indigena                             210
Raizal del archipielago               19
Palenquero                             2
Gitano (Rom)                           1
Name: count, dtype: int64

In [11]:
etnia = {
    'Gitano (Rom)': 'Otra etnia',
    'Raizal del archipielago':'Otra etnia',
    'Palenquero':'Otra etnia',
    }

In [12]:
data['ETNIA'] = data['ETNIA'].replace(etnia)

In [13]:
data['ETNIA'].value_counts()

ETNIA
Ninguno de los anteriores           6161
Negro, mulato (afrodescendiente)     509
Indigena                             210
Otra etnia                            22
Name: count, dtype: int64

## DISCAPACIDADES
Para el tratamiento de las discapacidades, se va a consolidar en una única variable con la siguiente codificación:

**Sin discapacidad:**
Nivel 4 en todas las variables de dificultad.

**Discapacidad leve:**
Nivel 3 en una o dos variables de dificultad.
Nivel 2 en una variable de dificultad, sin presencia de Nivel 3 en ninguna variable.

**Discapacidad moderada:**
Nivel 3 en tres o más variables de dificultad.
Nivel 2 en dos o más variables de dificultad, con presencia de Nivel 3 en al menos una variable.

**Discapacidad grave:**
Nivel 1 en cualquier variable de dificultad.
Nivel 2 en tres o más variables de dificultad.

In [14]:
data['DIFICULTAD_OIR'].value_counts()

DIFICULTAD_OIR
Sin dificultad               6864
Si, con alguna dificultad      27
Si, con mucha dificultad       10
No puede hacerlo                1
Name: count, dtype: int64

In [15]:
disc_num = {
    'No puede hacerlo':1,
    'Si, con mucha dificultad':2,
    'Si, con alguna dificultad':3,
    'Sin dificultad':4}

In [16]:
discapacidades = ['DIFICULTAD_OIR',
                  'DIFICULTAD_HABLAR',
                  'DIFICULTAD_VER',
                  'DIFICULTAD_MOVER_SUBIR_BAJAR',
                  'DIFICULTAD_AGARRAR',
                  'DIFICULTAD_ENTENDER',
                  'DIFICULTAD_BANARSE_VESTIRSE',
                  'DIFICULTAD_RELACIONARSE']

In [17]:
for col in discapacidades:
    data[col] = data[col].replace(disc_num)

C:\Users\andre\AppData\Local\Temp\ipykernel_36936\2736953229.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace(disc_num)


In [18]:
all(data[col][1] == 4 for col in discapacidades)

True

In [19]:
def agrupar_dificultades(df):
  """
  Esta función agrupa las dificultades de una persona en las siguientes categorías:
  * Sin discapacidad
  * Discapacidad leve
  * Discapacidad moderada
  * Discapacidad grave

  Parámetros:
    df: Un DataFrame de Pandas que contiene las variables de dificultad.

  Retorno:
    Una serie de Pandas que contiene la categoría de discapacidad para cada persona.
  """

  # Inicialización de la variable que guarda la categoría de discapacidad
  discapacidad = []

  # Recorrido por cada fila del DataFrame
  for i in range(df.shape[0]):
    # Conteo de las apariciones de cada nivel de dificultad
    niveles = df.iloc[i,:].value_counts()
    for i in [1,2,3,4]:
        try:
            niveles[i]
        except: 
            niveles[i] = 0
        

    # Categorización de la discapacidad
    if niveles[4] == 8:
      discapacidad.append("Sin discapacidad")
    elif niveles[3] <= 2 and niveles[2] <= 1:
      discapacidad.append("Discapacidad leve")
    elif niveles[3] >= 3 or (niveles[3] >= 1 and niveles[2] >= 2):
      discapacidad.append("Discapacidad moderada")
    else:
      discapacidad.append("Discapacidad grave")

  return pd.Series(discapacidad, name="Discapacidad")

# Agrupación de las dificultades
data["DISCAPACIDAD"] = agrupar_dificultades(data[discapacidades])

In [20]:
data['DISCAPACIDAD'].value_counts()

DISCAPACIDAD
Sin discapacidad         6516
Discapacidad leve         373
Discapacidad moderada       9
Discapacidad grave          4
Name: count, dtype: int64

In [21]:
data['DISCAPACIDAD'].count()

6902

In [22]:
data = data.drop(columns=discapacidades)

## Mayor Nivel Educativo

In [23]:
data['MAYOR_NIVEL_EDUCATIVO'].value_counts()

MAYOR_NIVEL_EDUCATIVO
Media academica (Bachillerato clasico)    2796
Universitaria                             1367
Basica secundaria (6o - 9o)                854
Tecnica profesional                        790
Basica primaria (1o - 5o)                  350
Tecnologica                                318
Media tecnica (Bachillerato tecnico)       280
Especializacion                             76
Ninguno                                     33
Maestria                                    24
Normalista                                  13
Doctorado                                    1
Name: count, dtype: int64

In [24]:
educacion = {
    'Especializacion ':'Postgrado',
    'Maestria ':'Postgrado',
    'Doctorado ':'Postgrado'
}

In [25]:
data['MAYOR_NIVEL_EDUCATIVO'] = data['MAYOR_NIVEL_EDUCATIVO'].replace(educacion)

In [26]:
data['MAYOR_NIVEL_EDUCATIVO'].value_counts()

MAYOR_NIVEL_EDUCATIVO
Media academica (Bachillerato clasico)    2796
Universitaria                             1367
Basica secundaria (6o - 9o)                854
Tecnica profesional                        790
Basica primaria (1o - 5o)                  350
Tecnologica                                318
Media tecnica (Bachillerato tecnico)       280
Postgrado                                  101
Ninguno                                     33
Normalista                                  13
Name: count, dtype: int64

## GÉNERO

In [27]:
genero = {
    'Hombre trans':'Persona Trans',
    'Mujer trans': 'Persona Trans',
}

In [28]:
data['GENERO'] = data['GENERO'].replace(genero)

## TIPO DE SANITARIO

In [29]:
sanitario = {
    'Letrina': 'Otro',
    'Bajamar': 'Otro',
}

In [30]:
data['TIPO_SANITARIO'] = data['TIPO_SANITARIO'].replace(sanitario)

## Donde se obtiene el agua

In [31]:
agua = {
    'De pila pública':'Otro',
    'Carrotanque':'Otro',
    'Aguatero':'Otro'
    
}

In [32]:
data['DONDE_OBTIENE_AGUA'] = data['DONDE_OBTIENE_AGUA'].replace(agua)

## Financieros
Para los casos financieros, se van a descartar aquellas columnas rebundantes cómo lo son el caso, de ningún método financiero, no saben si tiene algún metodo financiero o si tiene algún tipo de método financiero distinto a los definidos desde la encuesta

In [33]:
data = data.drop(columns = ['HOGAR_TIENE_OTRO_FINANCIERO', 'HOGAR_TIENE_NINGUNO_DE_ESTOS', 'HOGAR_NO_SABE_FINANCIERO'])

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 46 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   DIRECTORIO                            6902 non-null   int64  
 1   ZONA_TERRITORIAL                      6902 non-null   object 
 2   FACTOR_EXPANSION                      6902 non-null   float64
 3   DPTO                                  6902 non-null   object 
 4   SEXO                                  6902 non-null   object 
 5   EDAD                                  6902 non-null   int64  
 6   PARENTESCO_JEFE_DE_HOGAR              6902 non-null   object 
 7   MADRE_RESIDE_HOGAR                    6902 non-null   object 
 8   PADRE_RESIDE_HOGAR                    6902 non-null   object 
 9   SE_CONSIDERA_CAMPESINO                6902 non-null   object 
 10  COMUNIDAD_ES_CAMPESINA                6902 non-null   object 
 11  ETNIA            

Se van a descartar adicionalmente, aquellas variables relacionadas con los materiales sobre el que está construido la vivienda.

In [35]:
data = data.drop(columns = ['MATERIAL_PAREDES','MATERIAL_PISOS'])

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 44 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   DIRECTORIO                            6902 non-null   int64  
 1   ZONA_TERRITORIAL                      6902 non-null   object 
 2   FACTOR_EXPANSION                      6902 non-null   float64
 3   DPTO                                  6902 non-null   object 
 4   SEXO                                  6902 non-null   object 
 5   EDAD                                  6902 non-null   int64  
 6   PARENTESCO_JEFE_DE_HOGAR              6902 non-null   object 
 7   MADRE_RESIDE_HOGAR                    6902 non-null   object 
 8   PADRE_RESIDE_HOGAR                    6902 non-null   object 
 9   SE_CONSIDERA_CAMPESINO                6902 non-null   object 
 10  COMUNIDAD_ES_CAMPESINA                6902 non-null   object 
 11  ETNIA            

In [37]:
path = r"C:\Users\andre\OneDrive\Escritorio\Proyecto de Grado\result\db_agrupado"
path_export = os.path.join(path, month +'_Agrupado' +'.csv')
data.to_csv(path_export, index=False, sep=';', encoding='latin1')

In [38]:
cat_variables = data.drop(columns='ACTIVIDAD_OCUPADA_ULTIMA_SEMANA').select_dtypes(include=['object']).columns.to_list()

In [39]:
import sys
sys.path.append('c:\\Users\\andre\\OneDrive\\Documentos\\Maestría\\Proyecto de Grado')

In [40]:
from data_module.pruebas import calculate_woe_iv

In [41]:
iv_df = pd.DataFrame(columns=['IV'])

# Calculate WOE and IV for each variable in the categorical dataframe
for column in cat_variables:
    woe, iv = calculate_woe_iv(data, column, 'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA')
    sum_iv = sum([i for i in iv.values()])
    iv_df.loc[column, 'IV'] = sum_iv

# Display the resulting dataframe
print("Resultados del Cálculo de IV")
display(iv_df.sort_values(by='IV', ascending=False))

print("Variables con IV mayor a 0.1, es decir, predictores aceptables")
display(iv_df[iv_df['IV'] > 0.1].sort_values(by='IV', ascending=False))

iv_columns = list(iv_df[iv_df['IV'] > 0.1].sort_values(by='IV', ascending=False).index)

Resultados del Cálculo de IV


,IV
PARENTESCO_JEFE_DE_HOGAR,0.281619
MADRE_RESIDE_HOGAR,0.21639
ESTADO_CIVIL,0.192598
DPTO,0.155438
PADRE_RESIDE_HOGAR,0.072577
TIPO_OCUPACION_VIVIENDA,0.064025
MAYOR_NIVEL_EDUCATIVO,0.050143
TIPO_VIVIENDA,0.036521
DONDE_OBTIENE_AGUA,0.027612
ACTUALMENTE_ESTUDIA,0.02448


Variables con IV mayor a 0.1, es decir, predictores aceptables


,IV
PARENTESCO_JEFE_DE_HOGAR,0.281619
MADRE_RESIDE_HOGAR,0.21639
ESTADO_CIVIL,0.192598
DPTO,0.155438
